In [24]:
import cv2 #import the library
import imutils
image = cv2.imread("input.jpg")#read the image
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)#resizing image and change height to 500
 
# convert the image to grayscale, blur it, and find edges in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
 
# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 1: Edge Detection


In [25]:
# find the contours in the edged image, keeping only the largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
# loop over the contours
for c in cnts:
     # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break
#sort the contour,drawing the contour on image
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
#from the above code we extracted the table from the image

STEP 2: Find contours of paper


In [26]:
import numpy as np

In [27]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [28]:
#scanning the image to remove the noise
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
 
# convert the warped image to grayscale
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
 
# show the original and scanned images
print("STEP 3: Apply perspective transform")
# cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imwrite("Scanned.jpg", imutils.resize(warped, height = 650,width=420))
cv2.waitKey(0)


STEP 3: Apply perspective transform


-1

In [29]:
n=cv2.imread("Scanned.jpg")
print(n.shape)

(651, 420, 3)


In [30]:
#function to crop every horizontal row from image and save it
def clip(a):
    idx=0
    cropped_dir_path="E:/Users/NM/envs/cv/ocraa/fa/a"
    for i in range(21):        
            i=a[:31,:]
            cv2.imwrite(cropped_dir_path+str(idx) + '.png',i)  
            idx+=1
            a=a[31:,:]

In [31]:
clip(n)

In [32]:
#extracting the small rectangle block from the horizontal cropped row
def v_clip(a):
    idx=0
    for i in range(7):        
            i=a[:,:60]
#             print(i.shape)
            cv2.imwrite(cropped_dir_path+str(idx) + '.png',i)  
            idx+=1
            a=a[:,60:]

In [33]:
#loop over all the horizontal cropped image and extracting the small rectangle
for i in range(21):
    j=str(i)
    path="E:/Users/NM/envs/cv/ocraa/fa/a"
    ver=cv2.imread(path+j+".png")
    cropped_dir_path="E:/Users/NM/envs/cv/ocraa/fa/A0/b"+str(i)
    v_clip(ver)

In [34]:
#reading every small rectangle,listing the rgb value of block and then flating three value to get the single value through pix_val_flat
from PIL import Image
small_part_path = "E:/Users/NM/envs/cv/ocraa/faltu/A0/b"
l=[]
for i in range(1,21):
    b=str(i)
    for j in range(1,7):
        im=Image.open(small_part_path+b+str(j)+".png","r")
        pix_val = list(im.getdata())
        pix_val_flat = [x for sets in pix_val for x in sets]
        value=0
        #after getting pixels value of image,adding all the value 
        for i in pix_val_flat:
             value+=i     
        #after addition,with checking and comparing the value,we can get whether it is black block or white block        
        if value>550000:
            l.append("A")#adding P or A to list based on the magnitude of value.
# #             print("A"+b)
        else:
#             print("P"+b+)
            l.append("P")
            

In [35]:
#making the small list of 6 entries inside the main list
n=6
final = [l[i * n:(i + 1) * n] for i in range((len(l) + n - 1) // n )]
print(final)

[['P', 'A', 'P', 'A', 'A', 'P'], ['A', 'P', 'A', 'P', 'A', 'A'], ['A', 'A', 'P', 'A', 'A', 'A'], ['P', 'A', 'A', 'A', 'A', 'A'], ['A', 'A', 'P', 'A', 'A', 'A'], ['A', 'P', 'A', 'A', 'P', 'A'], ['P', 'A', 'P', 'A', 'A', 'P'], ['A', 'P', 'A', 'P', 'A', 'P'], ['P', 'A', 'P', 'A', 'P', 'A'], ['A', 'P', 'A', 'P', 'A', 'P'], ['P', 'A', 'P', 'A', 'P', 'A'], ['A', 'P', 'A', 'P', 'A', 'P'], ['P', 'P', 'P', 'A', 'P', 'A'], ['P', 'A', 'P', 'P', 'A', 'P'], ['A', 'P', 'A', 'P', 'A', 'A'], ['P', 'A', 'P', 'A', 'P', 'A'], ['A', 'P', 'P', 'A', 'P', 'A'], ['P', 'P', 'A', 'P', 'A', 'P'], ['A', 'P', 'A', 'P', 'A', 'P'], ['P', 'A', 'P', 'A', 'P', 'A']]


In [36]:
#assigning the respective number to the small list and structuring it like the image after classification
dict={}
for i in range(1,21):
    dict[i]=final[i-1]
print(dict)    

{1: ['P', 'A', 'P', 'A', 'A', 'P'], 2: ['A', 'P', 'A', 'P', 'A', 'A'], 3: ['A', 'A', 'P', 'A', 'A', 'A'], 4: ['P', 'A', 'A', 'A', 'A', 'A'], 5: ['A', 'A', 'P', 'A', 'A', 'A'], 6: ['A', 'P', 'A', 'A', 'P', 'A'], 7: ['P', 'A', 'P', 'A', 'A', 'P'], 8: ['A', 'P', 'A', 'P', 'A', 'P'], 9: ['P', 'A', 'P', 'A', 'P', 'A'], 10: ['A', 'P', 'A', 'P', 'A', 'P'], 11: ['P', 'A', 'P', 'A', 'P', 'A'], 12: ['A', 'P', 'A', 'P', 'A', 'P'], 13: ['P', 'P', 'P', 'A', 'P', 'A'], 14: ['P', 'A', 'P', 'P', 'A', 'P'], 15: ['A', 'P', 'A', 'P', 'A', 'A'], 16: ['P', 'A', 'P', 'A', 'P', 'A'], 17: ['A', 'P', 'P', 'A', 'P', 'A'], 18: ['P', 'P', 'A', 'P', 'A', 'P'], 19: ['A', 'P', 'A', 'P', 'A', 'P'], 20: ['P', 'A', 'P', 'A', 'P', 'A']}
